In [2]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import snscrape.modules.twitter as sntwitter
import matplotlib.pyplot as plt
import seaborn as sns
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
os.chdir('/Users/alisdghnia/Desktop/School - Graduate/Statistical Programming for Business Analytics - Spring 2023/')

In [4]:
df = pd.read_csv('chatGPT.csv')
df.head(6)

,Date,Tweet,Url,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,https://twitter.com/heyandriadaily/status/1619...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...
1,2023-01-29 15:07:59+00:00,I've been waiting for ChatGPT to open back up...,https://twitter.com/_doc_oc/status/16197139759...,_doc_oc,2022-02-08 01:15:45+00:00,False,24,144,0,0,Texas Medical Center,☢️ B.S. Nuclear Engineering \n🩺⚙️ Texas A&M E...
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,https://twitter.com/nikoonya/status/1619713889...,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,https://twitter.com/CSOonline/status/161971369...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys..."
4,2023-01-29 15:06:18+00:00,AI trainers = underpaid human annotators?!\n#C...,https://twitter.com/OmaymaS_/status/1619713552...,OmaymaS_,2009-12-13 15:58:13+00:00,False,1199,786,0,0,NaN,#datascience #machinelearning #rstats #python...
5,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,https://twitter.com/KevinPetschow/status/16197...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3798,4928,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...


In [5]:
# create a new column for the tweetID
df['tweetID'] = np.nan

# tweetID is the numbers after the last slash in the url
for i in range(len(df)):
    df['tweetID'][i] = df['Url'][i].split('/')[-1]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_6304/209259826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweetID'][i] = df['Url'][i].split('/')[-1]


# COLLECTING THE DATA

## DATA 1

In [5]:
data = []
for i, id in tqdm.tqdm(enumerate(df['tweetID']), total= len(df['tweetID'])):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            # new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
            new_data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

        data.append(new_data)

    if i > len(df['tweetID']):
        break

df2 = pd.DataFrame(data, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount', 
                                    'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

# add today's date to the dataframe
df2['date_scraped'] = pd.to_datetime('today').date()

# add a row to the dataframe in the beginning and assign null values since the first row is missing with the above code
df2.loc[-1] = np.nan
df2.index = df2.index + 1
df2 = df2.sort_index()

# save the dataframe as a csv file
df2.to_csv('chatGPT2.csv', index = False)

  0%|          | 0/179252 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_10487/1642198707.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
Unavailable user in card on tweet 1619643336276709376
User 1350034417931075585 not found in user refs in card on tweet 1619643336276709376
Unavailable user in card on tweet 1619428852392153090
User 1616528732407169027 not found in user refs in card on tweet 1619428852392153090
Unavailable user in card on tweet 1619399147131666433
User 1530435498 not found in user refs in card on tweet 1619399147131666433
Unavailable user in card on tweet 1619316359711670273
User 1616528732407169027 not found in user refs in card on tweet 1619316359711670273
Unavailable user in card on tweet 1619061929296949248
User 3067393243 not found in user refs in card on tweet 1619061929296949248
Unavailable user in card on tweet 1618993380545

KeyboardInterrupt: 

In [7]:
len(data)

126074

In [8]:
test_data = pd.DataFrame(data, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount',
                                          'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

In [10]:
test_data.tail()

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
126069,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126070,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126071,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126072,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126073,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
test_data.to_csv('test_data.csv')

## DATA 2

In [18]:
data2 = []
for i, id in tqdm.tqdm(enumerate(df['tweetID'][126074:]), total= len(df['tweetID'])-126074):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            # new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
            new_data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

        data2.append(new_data)

    if i > len(df['tweetID'])-126074:
        break

  0%|          | 0/53178 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_10487/3789065891.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
Unavailable user in card on tweet 1603024513784221697
Unavailable user in card on tweet 1603024513784221697
User 873939764876943363 not found in user refs in card on tweet 1603024513784221697
User 873939764876943363 not found in user refs in card on tweet 1603024513784221697
Unavailable user in card on tweet 1602695056523149318
User 1595487472297062400 not found in user refs in card on tweet 1602695056523149318
Unavailable user in card on tweet 1601921649820958721
User 3067393243 not found in user refs in card on tweet 1601921649820958721
Unavailable user in card on tweet 1601239597144764416
User 3067393243 not found in user refs in card on tweet 1601239597144764416


KeyError: 'result'

In [19]:
test_data2 = pd.DataFrame(data2, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount',
                                            'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

In [26]:
test_data2.to_csv('test_data2.csv')

In [27]:
len(data2) + len(data)

154084

## DATA 3

In [5]:
data3 = []
for i, id in tqdm.tqdm(enumerate(df['tweetID'][154084:]), total= len(df['tweetID'])-154084):
    for j in sntwitter.TwitterTweetScraper(tweetId = id).get_items():
        try:
            new_data = [j.date, j.rawContent, j.user.username, j.user.created, j.user.verified, j.user.followersCount, j.user.friendsCount,
                        j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]
        
        except Exception as e:
            # new_data = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
            new_data = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

        data3.append(new_data)

    if i > len(df['tweetID'])-154084:
        break

  0%|          | 0/25168 [00:00<?, ?it/s]

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_41331/1696863770.py:6: DeprecatedFeatureWarning: description is deprecated, use renderedDescription instead
  j.retweetCount, j.likeCount, j.user.location, j.user.description, j.place.country, j.lang, j.replyCount, ]


KeyError: 'result'

In [29]:
test_data3 = pd.DataFrame(data3, columns = ['date', 'tweet', 'username', 'user_created', 'user_verified', 'followersCount', 'friendsCount',
                                            'retweetCount', 'likeCount', 'location', 'user_description', 'country', 'lang', 'replyCount'])

In [32]:
len(df)

179252

In [34]:
nullCount = test_data.date.isnull().sum() + test_data2.date.isnull().sum() + test_data3.date.isnull().sum()

In [36]:
# number of remaining tweets
len(df) - nullCount

4235

# DATA PREPROCESSING AND FEATURE ENGINEERING

In [6]:
def tweet_power(dataset):
    likes = dataset['likeCount'] * 30
    retweets = dataset['retweetCount'] * 20
    following_follower_ratio = dataset['friendsCount'] / dataset['followersCount']
    verified = dataset['user_verified'] * 100

    # dataset['power'] = likes + retweets + verified
    for i in following_follower_ratio:
        if i > 0.6:
            dataset['power_value'] = 0.5 * (likes + retweets + verified)
        else:
            dataset['power_value'] = likes + retweets + verified

In [7]:
df_test1 = pd.read_csv('test_data.csv')
df_test2 = pd.read_csv('test_data2.csv')

In [6]:
df_test1

,Unnamed: 0,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,United States,en,0.0
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,United States,en,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126069,126069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126070,126070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126071,126071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126072,126072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_test2

,Unnamed: 0,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28005,28005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28006,28006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28007,28007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28008,28008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# concatenate the two dataframes
df_test = pd.concat([df_test1, df_test2], ignore_index = True)

In [9]:
df_test

,Unnamed: 0,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,United States,en,0.0
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,United States,en,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154079,28005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154080,28006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154081,28007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154082,28008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# remove rows after 154084
df_og = df[:154084]

# drop tweetID and Url columns
df_og = df_og.drop(['tweetID', 'Url'], axis = 1)
df_og

,Date,Tweet,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...
1,2023-01-29 15:07:59+00:00,I've been waiting for ChatGPT to open back up...,_doc_oc,2022-02-08 01:15:45+00:00,False,24,144,0,0,Texas Medical Center,☢️ B.S. Nuclear Engineering \n🩺⚙️ Texas A&M E...
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys..."
4,2023-01-29 15:06:18+00:00,AI trainers = underpaid human annotators?!\n#C...,OmaymaS_,2009-12-13 15:58:13+00:00,False,1199,786,0,0,NaN,#datascience #machinelearning #rstats #python...
...,...,...,...,...,...,...,...,...,...,...,...
154079,2022-12-07 03:30:08+00:00,My first questions … #ChatGPT https://t.co/Zze...,anandavardhanan,2014-03-30 03:07:58+00:00,False,1732,126,0,0,Mumbai,"Links, likes, and RT's need not be endorsement..."
154080,2022-12-07 03:29:59+00:00,@anobelodisho @CanesDavid @wandering_gu @urete...,TylerSbrt,2019-11-13 06:08:36+00:00,False,1634,513,0,0,"San Diego, CA",#radonc and #cancer scientist @UCSanDiego. Qua...
154081,2022-12-07 03:29:29+00:00,One example of where the AI train could be hea...,davidbutlermd,2013-04-04 05:42:19+00:00,False,1744,1932,1,1,"Sacramento, CA",✾ A 'ridiculous optimist' about fixing US #hea...
154082,2022-12-07 03:29:23+00:00,I broke Chat GPT #OpenAIChat #ChatGPT https://...,whoisorin,2009-03-12 07:30:52+00:00,False,896,829,0,0,Chandler AZ,Founder https://t.co/8ZiOaU5vpz Inefficiency b...


In [13]:
# drop Unnamed: 0 column from df_test
df_test = df_test.drop(['Unnamed: 0'], axis = 1)
df_test

KeyError: "['Unnamed: 0'] not found in axis"

In [12]:
len(df_og) - df_test.isnull().sum()[1]

3427

# SIMPLE ML WITH METADATA

ACCURACY = 0.5175
PRECISION = 0.5091383812010444

## TEST DATASET

In [14]:
# get 100 rows from df_test that are not null
non_null_df_test = df_test[df_test.tweet.notnull()]
non_null_df_test = non_null_df_test.tail(200)
non_null_df_test['survive'] = 1
non_null_df_test.drop(columns=['country', 'lang', 'replyCount'], inplace=True)

# get 300 rows from df_test that are null
null_df_test = df_test[df_test.tweet.isnull()]
null_df_test = null_df_test.tail(200)
null_df_test_updated = df_og.iloc[null_df_test.index]
null_df_test_updated['survive'] = 0
null_df_test_updated.columns = list(non_null_df_test.columns)
null_df_test_updated

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_6304/3081957457.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_df_test_updated['survive'] = 0


,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survive
153877,2022-12-07 04:35:32+00:00,ChatGPT response for top 10 programming Langua...,gsarantech,2022-04-08 14:24:39+00:00,False,29,250,0,0,NaN,Working as Senior Technical Consultant at @tar...,0
153878,2022-12-07 04:35:23+00:00,@OpenAI Just quick experimentation to see if ...,George11z11,2016-05-19 20:03:43+00:00,False,12,121,0,0,NaN,NaN,0
153879,2022-12-07 04:35:12+00:00,@ios_dev_alb Me who used #ChatGPT LOL,FaridG022,2019-09-23 04:32:00+00:00,False,336,1117,0,2,Pakistan,"iOS Dev + Physique, age: 22, #IOSdev at Peek i...",0
153880,2022-12-07 04:35:07+00:00,AI #ChatGPT what is say about @QUAINETWORK app...,maikasuwa_david,2022-02-20 17:42:58+00:00,False,122,434,0,1,Nigeria 🇳🇬,Forex trader (Synthetic indices) 📈📉 and Crypto...,0
153881,2022-12-07 04:34:31+00:00,@srivatsanp24 Just gave a brilliant comparison...,srikumarks,2009-02-04 15:52:00+00:00,False,357,355,0,1,India,"math, music (https://t.co/cdVyRPFPB8), human a...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
154079,2022-12-07 03:30:08+00:00,My first questions … #ChatGPT https://t.co/Zze...,anandavardhanan,2014-03-30 03:07:58+00:00,False,1732,126,0,0,Mumbai,"Links, likes, and RT's need not be endorsement...",0
154080,2022-12-07 03:29:59+00:00,@anobelodisho @CanesDavid @wandering_gu @urete...,TylerSbrt,2019-11-13 06:08:36+00:00,False,1634,513,0,0,"San Diego, CA",#radonc and #cancer scientist @UCSanDiego. Qua...,0
154081,2022-12-07 03:29:29+00:00,One example of where the AI train could be hea...,davidbutlermd,2013-04-04 05:42:19+00:00,False,1744,1932,1,1,"Sacramento, CA",✾ A 'ridiculous optimist' about fixing US #hea...,0
154082,2022-12-07 03:29:23+00:00,I broke Chat GPT #OpenAIChat #ChatGPT https://...,whoisorin,2009-03-12 07:30:52+00:00,False,896,829,0,0,Chandler AZ,Founder https://t.co/8ZiOaU5vpz Inefficiency b...,0


In [15]:
# concatenate the two dataframes
test1 = pd.concat([non_null_df_test, null_df_test_updated], ignore_index = True)
test1

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survive
0,2022-12-08 22:57:31+00:00,"Have already used #ChatGPT to evaluate #code,...",roblew9013,2009-07-15 02:47:30+00:00,False,211.0,753.0,0.0,1.0,"San Francisco, CA",Sharing thoughts on: Life Science 🧬 Data Scien...,1
1,2022-12-08 22:51:07+00:00,Everyone who works in tech is about to have a ...,cantblockjo,2018-08-10 00:11:22+00:00,False,15.0,103.0,0.0,0.0,"London, England",Some things are better left unsaid.,1
2,2022-12-08 21:51:50+00:00,Just had #chatGPT churn out 12 blog posts for ...,roblew9013,2009-07-15 02:47:30+00:00,False,211.0,753.0,0.0,0.0,"San Francisco, CA",Sharing thoughts on: Life Science 🧬 Data Scien...,1
3,2022-12-08 21:32:55+00:00,Amazing endless uses for #ChatGPT !\n👇🏼,docmaarten,2009-07-02 21:53:21+00:00,False,485.0,489.0,0.0,0.0,Adelaide Australia,skill acquisition • motor learning • cognitive...,1
4,2022-12-08 21:15:03+00:00,#ChatGPT is doing all of my email replies from...,liamchampton,2011-07-04 21:21:28+00:00,False,800.0,517.0,0.0,4.0,"London, UK",Sr. Regional Cloud Advocate @Microsoft & @Auth...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
395,2022-12-07 03:30:08+00:00,My first questions … #ChatGPT https://t.co/Zze...,anandavardhanan,2014-03-30 03:07:58+00:00,False,1732.0,126.0,0.0,0.0,Mumbai,"Links, likes, and RT's need not be endorsement...",0
396,2022-12-07 03:29:59+00:00,@anobelodisho @CanesDavid @wandering_gu @urete...,TylerSbrt,2019-11-13 06:08:36+00:00,False,1634.0,513.0,0.0,0.0,"San Diego, CA",#radonc and #cancer scientist @UCSanDiego. Qua...,0
397,2022-12-07 03:29:29+00:00,One example of where the AI train could be hea...,davidbutlermd,2013-04-04 05:42:19+00:00,False,1744.0,1932.0,1.0,1.0,"Sacramento, CA",✾ A 'ridiculous optimist' about fixing US #hea...,0
398,2022-12-07 03:29:23+00:00,I broke Chat GPT #OpenAIChat #ChatGPT https://...,whoisorin,2009-03-12 07:30:52+00:00,False,896.0,829.0,0.0,0.0,Chandler AZ,Founder https://t.co/8ZiOaU5vpz Inefficiency b...,0


## TRAIN DATASET

In [16]:
# get rows that are not null from df_test
df2 = df_test[df_test.tweet.notnull()]

df2 = df2.head(500)
df2

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,United States,en,0.0
4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,United States,en,0.0
87,2023-01-29 14:39:11+00:00,"Excellent piece.""Much of what is taught in #ST...",didem_un_ates,2009-09-01 14:19:17+00:00,False,1699.0,1433.0,4.0,9.0,"London, England",@Microsoft #Data #AI #Metaverse l #Responsible...,United Kingdom,en,0.0
93,2023-01-29 14:35:59+00:00,"#ChatGPT gets it… solve cons from list A, with...",UvaVault,2022-07-22 00:49:21+00:00,False,343.0,246.0,0.0,6.0,"Atlanta, GA",750ml Sold Out. Asset-backed NFT🍷+ Cellaring +...,United States,en,1.0
107,2023-01-29 14:30:14+00:00,“If you’re doing original reporting and uneart...,Joab_Jackson,2009-03-07 15:56:20+00:00,False,7110.0,6719.0,0.0,0.0,Bed-Stuy NYC / Sleepy Creek WV,Listening to the Technology / Editor-in-Chief ...,United States,en,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19377,2023-01-24 15:15:02+00:00,I've used #ChatGPT creatively for my clients. ...,SanchezMoses,2012-05-23 21:29:56+00:00,False,10759.0,818.0,1.0,1.0,"Phoenix, AZ",Child of God | Husband & Abuelito | Fmr High S...,United States,en,0.0
19379,2023-01-24 15:14:48+00:00,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,saadnajeebsaad,2014-07-28 09:38:57+00:00,False,91.0,558.0,0.0,0.0,NaN,NaN,Pakistan,en,0.0
19390,2023-01-24 15:13:03+00:00,Hello #ContentCreator I’ve generated a simple ...,jasonvestri,2009-10-08 23:55:05+00:00,False,146.0,406.0,0.0,0.0,"Austin, TX","UX & Product, #Google Cloud // #Apple, #Autode...",United States,en,2.0
19402,2023-01-24 15:11:23+00:00,@waitbutwhy: it’s playing out as you wrote 8 y...,theAlexW,2009-10-15 23:21:46+00:00,False,4787.0,73.0,0.0,0.0,LA,Astrophotographer by night (IG: bit.ly/3jRUXlj...,United States,en,0.0


In [17]:
# get rows that are null from df_test
df3 = df_test[df_test.tweet.isnull()]

df3 = df3.head(500)
df3

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# get index numbers from df3
df3_indx = df3.index

# get rows from df_og with the index numbers from df3_indx
df3_og = df_og.iloc[df3_indx]
df3_og

,Date,Tweet,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys..."
5,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3798,4928,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...
6,2023-01-29 15:05:51+00:00,"#ChatGPT wrote a poem about fatherhood, in the...",mitchcarrtv,2009-05-31 23:10:16+00:00,True,3792,946,0,0,"Chicago, IL",Anchor of Early Morning on @NewsNation | Luca’...
...,...,...,...,...,...,...,...,...,...,...,...
511,2023-01-29 11:23:57+00:00,OpenAI's #ChatGPT is writing a new book.\n\n👇 ...,TheEternalStory,2023-01-18 22:19:18+00:00,False,0,0,0,0,NaN,Asla bitmeyen bir kitap.
512,2023-01-29 11:23:56+00:00,I am getting more impressed by #ChatGPT’s abil...,weldeiry,2008-04-27 23:44:29+00:00,False,10651,9241,1,4,Pennsylvania,Physician-Scientist; Med-Onc; Pubs https://t.c...
513,2023-01-29 11:22:52+00:00,#ChatGPT is better than human authors.\n\n👇 Click,DexAuctor,2023-01-20 06:55:44+00:00,False,1,0,0,1,NaN,"Co-writer of the Twitter-based book ""The Etern..."
514,2023-01-29 11:22:41+00:00,Google announced a new ai chatGPT application....,NareshAskani,2017-02-28 08:27:59+00:00,False,15,51,0,2,"Hyderabad, India","Son of agriculture man, Thinker, Developer, D..."


In [19]:
df2.drop(columns=['country', 'lang', 'replyCount'], inplace=True)
df2

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description
1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...
4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...
87,2023-01-29 14:39:11+00:00,"Excellent piece.""Much of what is taught in #ST...",didem_un_ates,2009-09-01 14:19:17+00:00,False,1699.0,1433.0,4.0,9.0,"London, England",@Microsoft #Data #AI #Metaverse l #Responsible...
93,2023-01-29 14:35:59+00:00,"#ChatGPT gets it… solve cons from list A, with...",UvaVault,2022-07-22 00:49:21+00:00,False,343.0,246.0,0.0,6.0,"Atlanta, GA",750ml Sold Out. Asset-backed NFT🍷+ Cellaring +...
107,2023-01-29 14:30:14+00:00,“If you’re doing original reporting and uneart...,Joab_Jackson,2009-03-07 15:56:20+00:00,False,7110.0,6719.0,0.0,0.0,Bed-Stuy NYC / Sleepy Creek WV,Listening to the Technology / Editor-in-Chief ...
...,...,...,...,...,...,...,...,...,...,...,...
19377,2023-01-24 15:15:02+00:00,I've used #ChatGPT creatively for my clients. ...,SanchezMoses,2012-05-23 21:29:56+00:00,False,10759.0,818.0,1.0,1.0,"Phoenix, AZ",Child of God | Husband & Abuelito | Fmr High S...
19379,2023-01-24 15:14:48+00:00,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,saadnajeebsaad,2014-07-28 09:38:57+00:00,False,91.0,558.0,0.0,0.0,NaN,NaN
19390,2023-01-24 15:13:03+00:00,Hello #ContentCreator I’ve generated a simple ...,jasonvestri,2009-10-08 23:55:05+00:00,False,146.0,406.0,0.0,0.0,"Austin, TX","UX & Product, #Google Cloud // #Apple, #Autode..."
19402,2023-01-24 15:11:23+00:00,@waitbutwhy: it’s playing out as you wrote 8 y...,theAlexW,2009-10-15 23:21:46+00:00,False,4787.0,73.0,0.0,0.0,LA,Astrophotographer by night (IG: bit.ly/3jRUXlj...


In [20]:
df2['survived'] = 1
df2

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survived
1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,1
4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,1
87,2023-01-29 14:39:11+00:00,"Excellent piece.""Much of what is taught in #ST...",didem_un_ates,2009-09-01 14:19:17+00:00,False,1699.0,1433.0,4.0,9.0,"London, England",@Microsoft #Data #AI #Metaverse l #Responsible...,1
93,2023-01-29 14:35:59+00:00,"#ChatGPT gets it… solve cons from list A, with...",UvaVault,2022-07-22 00:49:21+00:00,False,343.0,246.0,0.0,6.0,"Atlanta, GA",750ml Sold Out. Asset-backed NFT🍷+ Cellaring +...,1
107,2023-01-29 14:30:14+00:00,“If you’re doing original reporting and uneart...,Joab_Jackson,2009-03-07 15:56:20+00:00,False,7110.0,6719.0,0.0,0.0,Bed-Stuy NYC / Sleepy Creek WV,Listening to the Technology / Editor-in-Chief ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
19377,2023-01-24 15:15:02+00:00,I've used #ChatGPT creatively for my clients. ...,SanchezMoses,2012-05-23 21:29:56+00:00,False,10759.0,818.0,1.0,1.0,"Phoenix, AZ",Child of God | Husband & Abuelito | Fmr High S...,1
19379,2023-01-24 15:14:48+00:00,#WeatherUpdate #Karachi #Khi #TempUpdate #Sunr...,saadnajeebsaad,2014-07-28 09:38:57+00:00,False,91.0,558.0,0.0,0.0,NaN,NaN,1
19390,2023-01-24 15:13:03+00:00,Hello #ContentCreator I’ve generated a simple ...,jasonvestri,2009-10-08 23:55:05+00:00,False,146.0,406.0,0.0,0.0,"Austin, TX","UX & Product, #Google Cloud // #Apple, #Autode...",1
19402,2023-01-24 15:11:23+00:00,@waitbutwhy: it’s playing out as you wrote 8 y...,theAlexW,2009-10-15 23:21:46+00:00,False,4787.0,73.0,0.0,0.0,LA,Astrophotographer by night (IG: bit.ly/3jRUXlj...,1


In [21]:
df3_og['survived'] = 0
df3_og

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_6304/2378352013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_og['survived'] = 0


,Date,Tweet,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription,survived
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...,0
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...,0
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys...",0
5,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3798,4928,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,0
6,2023-01-29 15:05:51+00:00,"#ChatGPT wrote a poem about fatherhood, in the...",mitchcarrtv,2009-05-31 23:10:16+00:00,True,3792,946,0,0,"Chicago, IL",Anchor of Early Morning on @NewsNation | Luca’...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
511,2023-01-29 11:23:57+00:00,OpenAI's #ChatGPT is writing a new book.\n\n👇 ...,TheEternalStory,2023-01-18 22:19:18+00:00,False,0,0,0,0,NaN,Asla bitmeyen bir kitap.,0
512,2023-01-29 11:23:56+00:00,I am getting more impressed by #ChatGPT’s abil...,weldeiry,2008-04-27 23:44:29+00:00,False,10651,9241,1,4,Pennsylvania,Physician-Scientist; Med-Onc; Pubs https://t.c...,0
513,2023-01-29 11:22:52+00:00,#ChatGPT is better than human authors.\n\n👇 Click,DexAuctor,2023-01-20 06:55:44+00:00,False,1,0,0,1,NaN,"Co-writer of the Twitter-based book ""The Etern...",0
514,2023-01-29 11:22:41+00:00,Google announced a new ai chatGPT application....,NareshAskani,2017-02-28 08:27:59+00:00,False,15,51,0,2,"Hyderabad, India","Son of agriculture man, Thinker, Developer, D...",0


## SIMPLE MODELS WITH METADATA

### PREPARING THE TRAIN AND TEST DATA

In [22]:
df3_og.columns = list(df2.columns)

In [23]:
# concatenate the two dataframes
X_train = pd.concat([df2, df3_og], ignore_index = True)

In [24]:
y_train = X_train.survived

In [27]:
X_train.drop(columns=['date', 'tweet', 'username', 'user_created', 'location', 'user_description', 'survived'], inplace=True)
tweet_power(X_train)

X_train.power_value = X_train.power_value.astype('float64')

In [28]:
X_train.user_verified = X_train.user_verified.astype('bool')

In [29]:
cols = ['user_verified', 'followersCount', 'friendsCount', 'retweetCount', 'likeCount']

In [31]:
test1.to_csv('Processed_Testing_Data.csv')

In [151]:
X_test = test1[cols]
X_test.user_verified = X_test.user_verified.astype('bool')

tweet_power(X_test)
X_test.power_value = X_test.power_value.astype('float64')

/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_24855/3620843244.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.user_verified = X_test.user_verified.astype('bool')
/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_24855/2985771660.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['power_value'] = 0.5 * (likes + retweets + verified)
/var/folders/_9/qq7pw_fx125c812gfv1wl_180000gn/T/ipykernel_24855/2985771660.py:12: SettingWithCopyWarning: 
A value is trying to be set

In [152]:
y_test = test1.survive

### ML MODELS

In [153]:
# import the necessary modules
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score

In [154]:
LR = LogisticRegression()
LR.fit(X_train, y_train)

y_pred = LR.predict(X_test)

accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.17      0.27       200
           1       0.53      0.94      0.68       200

    accuracy                           0.55       400
   macro avg       0.62      0.55      0.47       400
weighted avg       0.62      0.55      0.47       400



In [155]:
# import the necessary modules
from sklearn.ensemble import RandomForestClassifier

In [156]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)

y_pred = LR.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))

confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))

0.55
0.5282485875706214
              precision    recall  f1-score   support

           0       0.72      0.17      0.27       200
           1       0.53      0.94      0.68       200

    accuracy                           0.55       400
   macro avg       0.62      0.55      0.47       400
weighted avg       0.62      0.55      0.47       400



In [157]:
# import the necessary modules for xgboost
from xgboost import XGBClassifier

In [158]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_verified   4000 non-null   bool   
 1   followersCount  4000 non-null   float64
 2   friendsCount    4000 non-null   float64
 3   retweetCount    4000 non-null   float64
 4   likeCount       4000 non-null   float64
 5   power_value     4000 non-null   float64
dtypes: bool(1), float64(5)
memory usage: 160.3 KB


In [159]:
XGB = XGBClassifier()
XGB.fit(X_train, y_train)

y_pred = XGB.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))

confusion_matrix(y_test, y_pred)

print(classification_report(y_test, y_pred))

0.6175
0.6192893401015228
              precision    recall  f1-score   support

           0       0.62      0.62      0.62       200
           1       0.62      0.61      0.61       200

    accuracy                           0.62       400
   macro avg       0.62      0.62      0.62       400
weighted avg       0.62      0.62      0.62       400



# TOPIC MODELING WITH CHATGPT

In [186]:
import openai
from getpass import getpass
import tqdm.notebook as tqdm

In [167]:
openai.api_key = getpass()

In [168]:
model_engine = "text-davinci-003"
# model_engine = "text-embedding-ada-002"

In [169]:
df_og.head(10)

,Date,Tweet,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Location,UserDescription
0,2023-01-29 15:08:03+00:00,My hubby keeps saying I should try #ChatGPT.\n...,heyandriadaily,2021-11-18 00:15:55+00:00,False,1,0,0,0,USA,Intentional living - occasional luxuries - hea...
1,2023-01-29 15:07:59+00:00,I've been waiting for ChatGPT to open back up...,_doc_oc,2022-02-08 01:15:45+00:00,False,24,144,0,0,Texas Medical Center,☢️ B.S. Nuclear Engineering \n🩺⚙️ Texas A&M E...
2,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,311,167,0,0,"Kansas City, MO",Cofounder @uvavault | Multi-collector of @veef...
3,2023-01-29 15:06:53+00:00,Tune in to our next Episode: Bans and regulati...,CSOonline,2009-03-16 12:25:05+00:00,True,62409,816,0,0,Boston area,"From @FoundryIDG -- #CSO provides news, analys..."
4,2023-01-29 15:06:18+00:00,AI trainers = underpaid human annotators?!\n#C...,OmaymaS_,2009-12-13 15:58:13+00:00,False,1199,786,0,0,NaN,#datascience #machinelearning #rstats #python...
5,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3798,4928,0,0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...
6,2023-01-29 15:05:51+00:00,"#ChatGPT wrote a poem about fatherhood, in the...",mitchcarrtv,2009-05-31 23:10:16+00:00,True,3792,946,0,0,"Chicago, IL",Anchor of Early Morning on @NewsNation | Luca’...
7,2023-01-29 15:05:43+00:00,"Who is using #ChatGPT to generate caption, twe...",amar_mortal,2010-07-14 13:42:08+00:00,False,519,642,0,0,"Nasik, India",#F1 #techfreak #MUFC fan till i die #hakunamat...
8,2023-01-29 15:04:54+00:00,Are you looking for ways to boost your product...,canonnist,2016-05-23 15:22:47+00:00,False,40,53,0,0,NaN,Magician on retirement.
9,2023-01-29 15:04:50+00:00,"Educational Technology, Engagement, Learning a...",joedale,2007-05-10 06:36:14+00:00,False,34630,38077,1,0,Isle of Wight,UK Independent modern foreign languages & tech...


In [170]:
df_og.Tweet[:5]

0    My hubby keeps saying I should try #ChatGPT.\n...
1    I've been waiting for  ChatGPT to open back up...
2          #ChatGPT thank u for my pitch deck bullets.
3    Tune in to our next Episode: Bans and regulati...
4    AI trainers = underpaid human annotators?!\n#C...
Name: Tweet, dtype: object

In [183]:
train_df = pd.concat([df2, df3_og], ignore_index = True)
train_df

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survived
0,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,1
1,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,1
2,2023-01-29 14:39:11+00:00,"Excellent piece.""Much of what is taught in #ST...",didem_un_ates,2009-09-01 14:19:17+00:00,False,1699.0,1433.0,4.0,9.0,"London, England",@Microsoft #Data #AI #Metaverse l #Responsible...,1
3,2023-01-29 14:35:59+00:00,"#ChatGPT gets it… solve cons from list A, with...",UvaVault,2022-07-22 00:49:21+00:00,False,343.0,246.0,0.0,6.0,"Atlanta, GA",750ml Sold Out. Asset-backed NFT🍷+ Cellaring +...,1
4,2023-01-29 14:30:14+00:00,“If you’re doing original reporting and uneart...,Joab_Jackson,2009-03-07 15:56:20+00:00,False,7110.0,6719.0,0.0,0.0,Bed-Stuy NYC / Sleepy Creek WV,Listening to the Technology / Editor-in-Chief ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-01-29 11:23:57+00:00,OpenAI's #ChatGPT is writing a new book.\n\n👇 ...,TheEternalStory,2023-01-18 22:19:18+00:00,False,0.0,0.0,0.0,0.0,NaN,Asla bitmeyen bir kitap.,0
996,2023-01-29 11:23:56+00:00,I am getting more impressed by #ChatGPT’s abil...,weldeiry,2008-04-27 23:44:29+00:00,False,10651.0,9241.0,1.0,4.0,Pennsylvania,Physician-Scientist; Med-Onc; Pubs https://t.c...,0
997,2023-01-29 11:22:52+00:00,#ChatGPT is better than human authors.\n\n👇 Click,DexAuctor,2023-01-20 06:55:44+00:00,False,1.0,0.0,0.0,1.0,NaN,"Co-writer of the Twitter-based book ""The Etern...",0
998,2023-01-29 11:22:41+00:00,Google announced a new ai chatGPT application....,NareshAskani,2017-02-28 08:27:59+00:00,False,15.0,51.0,0.0,2.0,"Hyderabad, India","Son of agriculture man, Thinker, Developer, D...",0


In [214]:
test1

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,survive
0,2022-12-08 22:57:31+00:00,"Have already used #ChatGPT to evaluate #code,...",roblew9013,2009-07-15 02:47:30+00:00,False,211.0,753.0,0.0,1.0,"San Francisco, CA",Sharing thoughts on: Life Science 🧬 Data Scien...,1
1,2022-12-08 22:51:07+00:00,Everyone who works in tech is about to have a ...,cantblockjo,2018-08-10 00:11:22+00:00,False,15.0,103.0,0.0,0.0,"London, England",Some things are better left unsaid.,1
2,2022-12-08 21:51:50+00:00,Just had #chatGPT churn out 12 blog posts for ...,roblew9013,2009-07-15 02:47:30+00:00,False,211.0,753.0,0.0,0.0,"San Francisco, CA",Sharing thoughts on: Life Science 🧬 Data Scien...,1
3,2022-12-08 21:32:55+00:00,Amazing endless uses for #ChatGPT !\n👇🏼,docmaarten,2009-07-02 21:53:21+00:00,False,485.0,489.0,0.0,0.0,Adelaide Australia,skill acquisition • motor learning • cognitive...,1
4,2022-12-08 21:15:03+00:00,#ChatGPT is doing all of my email replies from...,liamchampton,2011-07-04 21:21:28+00:00,False,800.0,517.0,0.0,4.0,"London, UK",Sr. Regional Cloud Advocate @Microsoft & @Auth...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
395,2022-12-07 03:30:08+00:00,My first questions … #ChatGPT https://t.co/Zze...,anandavardhanan,2014-03-30 03:07:58+00:00,False,1732.0,126.0,0.0,0.0,Mumbai,"Links, likes, and RT's need not be endorsement...",0
396,2022-12-07 03:29:59+00:00,@anobelodisho @CanesDavid @wandering_gu @urete...,TylerSbrt,2019-11-13 06:08:36+00:00,False,1634.0,513.0,0.0,0.0,"San Diego, CA",#radonc and #cancer scientist @UCSanDiego. Qua...,0
397,2022-12-07 03:29:29+00:00,One example of where the AI train could be hea...,davidbutlermd,2013-04-04 05:42:19+00:00,False,1744.0,1932.0,1.0,1.0,"Sacramento, CA",✾ A 'ridiculous optimist' about fixing US #hea...,0
398,2022-12-07 03:29:23+00:00,I broke Chat GPT #OpenAIChat #ChatGPT https://...,whoisorin,2009-03-12 07:30:52+00:00,False,896.0,829.0,0.0,0.0,Chandler AZ,Founder https://t.co/8ZiOaU5vpz Inefficiency b...,0


In [215]:
categories = ['safety, business, literature, health, productivity, or other']

lst_test = []

for i, id in tqdm.tqdm(enumerate(test1.tweet), total = len(test1.tweet)):
    # print(id)
    try:
        output = openai.Completion.create(

            engine=model_engine,
            prompt= f"{id} \n for the tweet above give me the topic of the tweet. for the topic of the tweet, you can only choose from the following categories: {categories}. only output the category for the tweet.",
            max_tokens=500,
            temperature=0.5,

        )
    except Exception:
        pass

    # print(output['choices'][0]['text'])
    lst_test.append([id ,output['choices'][0]['text']])

# lst_test
lst_final = []
for i in range(len(lst_test)):
    lst_test[i][1] = lst_test[i][1].replace('\n', '')
    # print(i)
    # lst_final.append(lst_)

lst_test
# lst_final
# train_topics_df = pd.DataFrame(lst_test, columns = ['tweet', 'topic'])
test_topics_df = pd.DataFrame(lst_test, columns = ['tweet', 'topic'])

  0%|          | 0/400 [00:00<?, ?it/s]

In [216]:
# train_topics_df
test_topics_df

,tweet,topic
0,"Have already used #ChatGPT to evaluate #code,...",Productivity
1,Everyone who works in tech is about to have a ...,Productivity
2,Just had #chatGPT churn out 12 blog posts for ...,Productivity
3,Amazing endless uses for #ChatGPT !\n👇🏼,Other
4,#ChatGPT is doing all of my email replies from...,Productivity
...,...,...
395,My first questions … #ChatGPT https://t.co/Zze...,Other
396,@anobelodisho @CanesDavid @wandering_gu @urete...,Other
397,One example of where the AI train could be hea...,Health
398,I broke Chat GPT #OpenAIChat #ChatGPT https://...,Other


In [217]:
# train_topics_df.topic.value_counts()
test_topics_df.topic.value_counts()

Other           263
Productivity     75
other            19
Business         15
Safety           11
Literature       10
Health            5
Tennis            1
Environment       1
Name: topic, dtype: int64

In [218]:
# train_topics_df.to_csv('train_topics.csv', index = False)
test_topics_df.to_csv('test_topics.csv', index = False)

In [59]:
df_og.Tweet[0]

'My hubby keeps saying I should try #ChatGPT.\n\nI should’ve known it would be more complicated. He being a data scientist fluent in Python, sure he’d think it was way easier.'

In [ ]:
categories = ['casual, professional, funny']

In [66]:
output = openai.Completion.create(

    engine=model_engine,
    prompt= f"give me a list with the 5 following items and nothing more for each tweet: sentiment value, sentiment polarity, subjectivity value, subjectivity polarity, and the topic of the tweet, which describe this tweet {df_og.Tweet[:3]}. Also, for the topic column, you can only choose from the following categories: {categories}.",
    max_tokens=500,
    temperature=0,

)
output['choices'][0]['text']

'\n\nTweet 0: \nSentiment Value: 0, Sentiment Polarity: Neutral, Subjectivity Value: 0, Subjectivity Polarity: Neutral, Topic: Technology and Business\n\nTweet 1: \nSentiment Value: 0, Sentiment Polarity: Neutral, Subjectivity Value: 0, Subjectivity Polarity: Neutral, Topic: Technology and Safety\n\nTweet 2: \nSentiment Value: 0, Sentiment Polarity: Neutral, Subjectivity Value: 0, Subjectivity Polarity: Neutral, Topic: Technology and Productivity'

In [74]:
df_test

,date,tweet,username,user_created,user_verified,followersCount,friendsCount,retweetCount,likeCount,location,user_description,country,lang,replyCount
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-29 15:07:38+00:00,#ChatGPT thank u for my pitch deck bullets.,nikoonya,2011-01-10 16:24:18+00:00,False,339.0,214.0,0.0,1.0,"Kansas City, MO",🍷 Co-founder & Doc’s List @UvaVault | Holder o...,United States,en,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-29 15:06:16+00:00,#ChatGPT: ‘Bigger than anything’ #HR has ever ...,KevinPetschow,2008-02-02 15:28:48+00:00,False,3802.0,4933.0,0.0,0.0,"Tampa, FL",Strategic #PR #marketing & #digitalmedia #lead...,United States,en,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
random_indx = np.random.randint(0, len(df_test), 1200)
random_indx

array([ 93072,  80253, 144129, ...,  16875,  22667,  69027])